In [3]:
import os
import json
import time
from IPython.display import Markdown, display
import requests
import textwrap
from llama_index.llms import ChatMessage, OpenAI
import boto3
from utils import read_file, save_file
from dotenv import load_dotenv

In [26]:
load_dotenv()
analsis_gpt4_turbo_1 = read_file('reports/moody-risk-calc-analysis-gpt-4-turbo-1.md')
analsis_claude_1 = read_file('reports/moody-risk-calc-analysis-cloude-21-1.md')
moody_paper = read_file('data/whitepaper/riskcalc-3.1-whitepaper.txt')

In [7]:
br = boto3.client(service_name='bedrock')
model_summaries = br.list_foundation_models()['modelSummaries']
#print(json.dumps(model_summaries, indent=4))

In [28]:
def review_claude(document, question, analysis, model='anthropic.claude-v2:1', temperature=0, tokens=3000, top_p=0.9, top_k=250):
    instructions = f"""Human: Act as a reviewer to prvide effective challenge of model analysis document.
Focus on reviewing and challenging model analyssis document and not the model itself.
You are provided with model whitepaper, analysis objective, and analysis document to perform the peromrme effective challenge. 
Your response should use well formatted Markdown forman and include:
Bullet points highlighting specific observiation, concerns and limitation with direct quotations from the 
model analsisis document citing specific sections that support the challenge.
A clear, evidence-based sumary of finding and recommendations for improving analysis document.
<whitepaper>
{document}
</whitepaper>
<objective>
{question}
</objective>
<analysis>
{analysis}
Assistant:"""
    
    brt = boto3.client(service_name='bedrock-runtime')
    
    body = json.dumps({
    "prompt": instructions,
    "max_tokens_to_sample": tokens,
    "temperature": temperature,
    "top_p": top_p,
    "top_k": top_k
    })

    accept = 'application/json'
    contentType = 'application/json'

    response = brt.invoke_model(body=body, modelId=model, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())

    return response_body.get('completion')

In [23]:
def review(document, question, analysis, model='gpt-4', temperature=0, max_tokens=4096, top_p=0.5, frequency_penalty=0, presence_penalty=0): 
    instructions = f"""Act as a reviewer to prvide effective challenge of model analysis document.
Focus on reviewing and challenging model analyssis document and not the model itself.
You are provided with model whitepaper, analysis objective, and analysis document to perform the peromrme effective challenge. 
Your response should use well formatted Markdown forman and include:
Bullet points highlighting specific observiation, concerns and limitation with direct quotations from the 
model analsisis document citing specific sections that support the challenge.
A clear, evidence-based sumary of finding and recommendations for improving analysis document.
<whitepaper>
{document}
</whitepaper>
<objective>
{question}
</objective>
<analysis>
{analysis}
</analysis>"""
    
    
    messages = [
        ChatMessage(role="system", content=instructions)
        #ChatMessage(role="assistant", content="provide model whitepaper"),
        #ChatMessage(role="user", content=document),
        #ChatMessage(role="assistant", content="what was analysis objective"),
        #ChatMessage(role="user", content=question),
        #ChatMessage(role="assistant", content="provide analysis document"),
        #ChatMessage(role="user", content=analysis)
    ]

    llm = OpenAI(  
        api_key=os.environ.get('OPENAI_API_KEY'),  
        model=model, 
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty)  

    response = llm.chat(messages)
    return response.message.content

In [24]:
q = 'Identify any specific limitations or concerns regarding the use of the model in a stagflation environment'

content = review(moody_paper, q, analsis_gpt4_turbo_1, 'gpt-4-turbo-preview')
display(Markdown(content))

### Review and Challenge of Model Analysis Document

#### Observations and Concerns

- **Economic Cycle Sensitivity**: The analysis raises a valid concern about the model's sensitivity to economic cycles, particularly stagflation. However, it does not provide specific evidence from the model's documentation that directly addresses how the model has performed historically during similar economic conditions. This gap in the analysis could be addressed by requesting historical performance data during past periods of stagflation or economic downturns.

- **Utilization of Market-Based Information**: The concern regarding the reliability of market-based information during stagflation is well-founded. However, the analysis could benefit from a deeper dive into how the model's use of industry-level distance-to-default measures might be impacted specifically by stagflation. For instance, "The model also incorporates industry-specific and economy-wide market information..." could be further explored to understand the historical correlation between these measures and actual default rates during stagflation.

- **Data Quality and Relevance**: The analysis mentions concerns about data quality and relevance but does not cite specific sections of the model documentation that discuss how the model updates its database or handles rapidly changing economic conditions. A challenge could be to identify whether the model's data refresh rates and sources are robust enough to capture the nuances of stagflation.

- **Model Calibration and Validation**: While the analysis correctly identifies potential issues with model calibration and validation during stagflation, it stops short of challenging the model's existing validation processes. For example, "RiskCalc v3.1 models are designed to meet the Basel II Accord’s stringent requirements for validating ratings..." could be scrutinized to determine if these requirements adequately cover scenarios like stagflation.

- **Industry-Specific Effects**: The analysis points out that stagflation impacts industries differently and questions whether the model's industry adjustments are sufficient. A more detailed challenge could involve examining the model's methodology for incorporating industry-specific trends and default rates to assess its comprehensiveness and accuracy.

#### Summary of Findings and Recommendations

The analysis document provides a thoughtful examination of the potential limitations and concerns regarding the use of the RiskCalc v3.1 model in a stagflation environment. However, the analysis could be strengthened by:

- Requesting or referencing historical performance data of the model during past periods of economic stress similar to stagflation.
- Conducting a more detailed examination of how market-based information and industry-specific adjustments are utilized and validated within the model, especially in the context of rapidly changing economic conditions.
- Challenging the model's calibration and validation processes to ensure they are robust enough to handle unique economic scenarios like stagflation.
- Recommending specific enhancements or supplementary measures, such as incorporating additional economic indicators sensitive to stagflation, to improve the model's accuracy and relevance during such periods.

Overall, while the analysis raises important concerns, a more evidence-based approach, including direct citations from the model documentation and historical performance data, would provide a stronger foundation for the challenges and recommendations presented.

In [32]:
q = 'Identify any specific limitations or concerns regarding the use of the model in a stagflation environment'

content = review(moody_paper, q, analsis_claude_1, 'gpt-4-turbo-preview')
display(Markdown(content))

### Review and Challenge of Model Analysis Document

#### Observations and Concerns:

- **Reliance on Market Data**: The model's incorporation of market data at the industry level to inform the distance-to-default measure could be a limitation in a stagflation environment. Market data might become less reliable if financial markets experience high volatility and uncertainty, which are common in stagflation scenarios. This concern is indirectly supported by the discussion in Section 3.2, where the importance of market-based information is highlighted.

- **Historical Data and Unprecedented Scenarios**: The model's development and validation include data from volatile periods such as 2000-2002, which is commendable. However, stagflation presents a unique combination of high inflation and stagnant economic growth that may not be fully represented in historical data sets. The model's performance in such unprecedented scenarios could be uncertain, as indicated in the potential limitations section of the analysis document.

- **Inflation Data Integration**: The analysis document suggests that direct inclusion of inflation data could potentially enhance the model's responsiveness to stagflation. This implies a limitation in the current model version, where explicit inflation metrics might not be directly accounted for, potentially affecting its predictive accuracy in high inflation environments.

#### Summary of Findings and Recommendations:

The RiskCalc v3.1 model demonstrates robust development, extensive validation, and the ability to stress test across various economic conditions, making it a valuable tool for assessing credit risk. However, specific limitations have been identified concerning its application in a stagflation environment, primarily related to its reliance on market data and the potential for unprecedented economic scenarios not fully captured in historical data.

**Recommendations for Improving the Analysis Document**:

1. **Enhance Transparency on Market Data Reliability**: Provide a more detailed discussion on how market data volatility, especially in stagflation scenarios, could impact the model's distance-to-default measure and overall predictive accuracy.

2. **Incorporate Inflation Metrics**: Explore the feasibility of directly integrating inflation data into the model to enhance its responsiveness to stagflation. This could involve developing additional model variants or adjustments that account for inflationary pressures.

3. **Expand Stress Testing Scenarios**: Include stress testing for hypothetical stagflation scenarios that go beyond historical data, using simulated data sets that capture extreme combinations of high inflation and low growth. This could help assess the model's robustness in unprecedented economic conditions.

4. **Ongoing Model Monitoring and Updates**: Establish a framework for continuous monitoring of the model's performance across various economic cycles, with a particular focus on stagflation-like conditions. This should include regular updates to the model based on new economic data and insights.

By addressing these recommendations, the analysis document can provide a more comprehensive evaluation of the RiskCalc v3.1 model's applicability in stagflation environments, enhancing its utility for decision-makers.